In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import matplotlib.ticker as plticker
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [7]:
#Importando as informações

copas = pd.read_csv('Copa_puro.csv', sep=';', encoding = "ISO-8859-1")
partidas = pd.read_csv('Partidas_puro.csv', sep=';', encoding = "ISO-8859-1")
jogadores = pd.read_csv('Jogadores_puro.csv', sep=';', encoding = "ISO-8859-1")

In [10]:
#Verificando informações

copas.head()

,Ano,Pais,Campeao,Vice-Campeao,3 Colocado,4 Colocado,Total de Gols,Total de Equipes,Total de Partidas,Publico
0,1930,Uruguay,Uruguay,Argentina,USA,Yugoslavia,70,13,18,590.549
1,1934,Italy,Italy,Czechoslovakia,Germany,Austria,70,16,17,363.000
2,1938,France,Italy,Hungary,Brazil,Sweden,84,15,18,375.700
3,1950,Brazil,Uruguay,Brazil,Sweden,Spain,88,13,22,1.045.246
4,1954,Switzerland,Germany FR,Hungary,Austria,Uruguay,140,16,26,768.607


In [88]:
partidas.head()

,Ano,Data,Fase,Estadio,Cidade,Casa,GolsCasa,GolsFora,Fora,Observacao,...,Gols Casa 1 Tempo,Gols Fora 1 tempo,Arbitro,Auxiliar 1,Auxiliar 2,RodadaID,PartidaID,Sigla Time Casa,Sigla Time Fora,vencedor
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,3,0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201,1096,FRA,MEX,2
1,1930,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3,0,Belgium,,...,2,0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201,1090,USA,BEL,2
2,1930,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,,...,2,0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201,1093,YUG,BRA,2
3,1930,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3,1,Peru,,...,1,0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201,1098,ROU,PER,2
4,1930,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1,0,France,,...,0,0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201,1085,ARG,FRA,2


In [12]:
jogadores.head()

,RodadaID,PartidaID,Time,Coach,Principal,Numero,Nome,Posicao,Eventos
0,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Alex THEPOT,GK,NaN
1,201,1096,MEX,LUQUE Juan (MEX),S,0,Oscar BONFIGLIO,GK,NaN
2,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Marcel LANGILLER,NaN,G40'
3,201,1096,MEX,LUQUE Juan (MEX),S,0,Juan CARRENO,NaN,G70'
4,201,1096,FRA,CAUDRON Raoul (FRA),S,0,Ernest LIBERATI,NaN,NaN


In [254]:
#Criando coluna com time vencedor
vencedor = []
gols = []
for i in range (len(partidas['Casa'])):
    if partidas['GolsCasa'][i] > partidas['GolsFora'][i]:
        vencedor.append('2')
        gols.append(partidas['GolsCasa'][i])
    elif partidas['GolsCasa'][i] < partidas['GolsFora'][i]:
        vencedor.append('0')
        gols.append('0')
    else:
        vencedor.append('1')
        gols.append(partidas['GolsFora'][i])
partidas['vencedor'] = vencedor
partidas['gols'] = gols
partidas.head()

,Ano,Data,Fase,Estadio,Cidade,Casa,GolsCasa,GolsFora,Fora,Observacao,...,Gols Fora 1 tempo,Arbitro,Auxiliar 1,Auxiliar 2,RodadaID,PartidaID,Sigla Time Casa,Sigla Time Fora,vencedor,gols
0,1930,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,...,0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201,1096,FRA,MEX,2,4
1,1930,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3,0,Belgium,,...,0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201,1090,USA,BEL,2,3
2,1930,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,,...,0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201,1093,YUG,BRA,2,2
3,1930,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3,1,Peru,,...,0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201,1098,ROU,PER,2,3
4,1930,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1,0,France,,...,0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201,1085,ARG,FRA,2,1


In [255]:
jogos_br = partidas[(partidas['Casa'] == 'Brazil') | (partidas['Fora'] == 'Brazil')]
brasil = jogos_br.iloc[:]
brasil.head()

,Ano,Data,Fase,Estadio,Cidade,Casa,GolsCasa,GolsFora,Fora,Observacao,...,Gols Fora 1 tempo,Arbitro,Auxiliar 1,Auxiliar 2,RodadaID,PartidaID,Sigla Time Casa,Sigla Time Fora,vencedor,gols
2,1930,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,,...,0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201,1093,YUG,BRA,2,2
11,1930,20 Jul 1930 - 13:00,Group 2,Estadio Centenario,Montevideo,Brazil,4,0,Bolivia,,...,0,BALWAY Thomas (FRA),MATEUCCI Francisco (URU),VALLEJO Gaspar (MEX),201,1091,BRA,BOL,2,4
23,1934,27 May 1934 - 16:30,Preliminary round,Luigi Ferraris,Genoa,Spain,3,1,Brazil,,...,0,BIRLEM Alfred (GER),CARMINATI Ettore (ITA),IVANCSICS Mihaly (HUN),204,1111,ESP,BRA,2,3
40,1938,05 Jun 1938 - 17:30,First round,Stade de la Meinau,Strasbourg,Brazil,6,5,Poland,Brazil win after extra time,...,0,EKLIND Ivan (SWE),POISSANT Louis (FRA),KISSENBERGER Ernest (FRA),206,1150,BRA,POL,2,6
44,1938,12 Jun 1938 - 17:00,Quarter-finals,Stade du Parc Lescure,Bordeaux,Brazil,1,1,Czechoslovakia,,...,0,VON HERTZKA Pal (HUN),SCARPI Giuseppe (ITA),DE LA SALLE Charles (FRA),429,1152,BRA,TCH,1,1


In [256]:
dados_finais = partidas.filter(items=['Casa','Fora','gols','vencedor'])

dados_finais = pd.get_dummies(dados_finais, prefix=['Casa', 'Fora'], columns=['Casa', 'Fora'])

modelo_dados = dados_finais.drop(['gols','vencedor'], axis=1)
modelo_resultado = dados_finais['vencedor']
modelo_resultado = modelo_resultado.astype('int')

modelo_resultado_gols = dados_finais['gols']
modelo_resultado_gols = modelo_resultado_gols.astype('int')

X_treino, X_teste, Y_treino, Y_teste = train_test_split(modelo_dados, modelo_resultado, test_size=0.30, random_state=13)

In [257]:
Y_treino.tail()

742    1
528    0
74     2
176    2
338    2
Name: vencedor, dtype: int32

In [258]:
modelo_resultado.tail()

847    1
848    0
849    1
850    0
851    2
Name: vencedor, dtype: int32

In [259]:
lr = LogisticRegression()
lr.fit(X_treino, Y_treino)
score = lr.score(X_treino, Y_treino)
score_2 = lr.score(X_teste, Y_teste)

print("Treino acuracia: ", '%.3f'%(score))
print("Teste acuracia: ", '%3.f' %(score_2))

Treino acuracia:  0.678
Teste acuracia:    1


In [260]:
Y_teste.tail()

750    0
105    2
676    0
45     2
675    2
Name: vencedor, dtype: int32

In [261]:
copa_2018 = pd.read_csv('copa_2018_predicao.csv')

In [262]:
copa_2018.head()

,Round Number,Date,Location,Casa,Fora,Group,Result
0,1,14/06/2018 18:00,"Luzhniki Stadium, Moscow",Russia,Saudi Arabia,Group A,NaN
1,1,15/06/2018 15:00,Ekaterinburg Stadium,Egypt,Uruguay,Group A,NaN
2,1,15/06/2018 18:00,Saint Petersburg Stadium,Morocco,Iran,Group B,NaN
3,1,15/06/2018 21:00,"Fisht Stadium, Sochi",Portugal,Spain,Group B,NaN
4,1,16/06/2018 13:00,Kazan Arena,France,Australia,Group C,NaN


In [266]:
X2_teste_predicao = copa_2018.filter(items=['Casa','Fora'])
backup_X2_teste = copa_2018.filter(items=['Casa','Fora','Group'])
X2_teste_predicao = pd.get_dummies(X2_teste_predicao, prefix=['Casa','Fora'], columns=['Casa','Fora'])

colunas_faltantes = set(modelo_dados.columns) - set(X2_teste_predicao.columns)
for c in colunas_faltantes:
    X2_teste_predicao[c] = 0
X2_teste_predicao = X2_teste_predicao[modelo_dados.columns]

#X2_teste_predicao = X2_teste_predicao.drop(['vencedor'], axis=1)

X2_teste_predicao.head()

,Casa_Algeria,Casa_Angola,Casa_Argentina,Casa_Australia,Casa_Austria,Casa_Belgium,Casa_Bolivia,Casa_Brazil,Casa_Bulgaria,Casa_Cameroon,...,Fora_Ukraine,Fora_Uruguay,Fora_Wales,Fora_Yugoslavia,Fora_Zaire,"Fora_rn"">Bosnia and Herzegovina","Fora_rn"">Republic of Ireland","Fora_rn"">Serbia and Montenegro","Fora_rn"">Trinidad and Tobago","Fora_rn"">United Arab Emirates"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [267]:
predicoes = lr.predict(X2_teste_predicao)
for i in range(0, len(X2_teste_predicao)):
    print('Grupo: ' + backup_X2_teste.iloc[i,2])
    print(backup_X2_teste.iloc[i,0] + " x " + backup_X2_teste.iloc[i,1])
    if predicoes[i] == 2:
        print("Vencedor da partida: " + backup_X2_teste.iloc[i,0])
    elif predicoes[i] == 1:
        print("Empate")
    elif predicoes[i] == 0:
        print("Vencedor da partida: " + backup_X2_teste.iloc[i,1])    
    print('Probabilidade de ' + backup_X2_teste.iloc[i,0] + ' vencer: ', '%.3f'%(lr.predict_proba(X2_teste_predicao)[i][2]))
    print('Probabilidade de empate', '%.3f'%(lr.predict_proba(X2_teste_predicao)[i][1]))
    print('Probabilidade de ' + backup_X2_teste.iloc[i,1] + ' vencer: ', '%.3f'%(lr.predict_proba(X2_teste_predicao)[i][0]))
    print("")

Grupo: Group A
Russia x Saudi Arabia
Vencedor da partida: Russia
Probabilidade de Russia vencer:  0.723
Probabilidade de empate 0.178
Probabilidade de Saudi Arabia vencer:  0.100

Grupo: Group A
Egypt x Uruguay
Vencedor da partida: Egypt
Probabilidade de Egypt vencer:  0.510
Probabilidade de empate 0.220
Probabilidade de Uruguay vencer:  0.270

Grupo: Group B
Morocco x Iran
Vencedor da partida: Morocco
Probabilidade de Morocco vencer:  0.439
Probabilidade de empate 0.310
Probabilidade de Iran vencer:  0.251

Grupo: Group B
Portugal x Spain
Vencedor da partida: Portugal
Probabilidade de Portugal vencer:  0.489
Probabilidade de empate 0.067
Probabilidade de Spain vencer:  0.444

Grupo: Group C
France x Australia
Vencedor da partida: France
Probabilidade de France vencer:  0.729
Probabilidade de empate 0.222
Probabilidade de Australia vencer:  0.050

Grupo: Group D
Argentina x Iceland
Vencedor da partida: Argentina
Probabilidade de Argentina vencer:  0.713
Probabilidade de empate 0.153
Pr

In [222]:
for i in range(0, backup_X2_teste.shape[0]):
    print('Grupo: ' + backup_X2_teste.iloc[i,2])
    print(backup_X2_teste.iloc[i,0] + " x " + backup_X2_teste.iloc[i,1])
    if predicoes[i] == 2:
        print("Vencedor da partida: " + backup_X2_teste.iloc[i,0])
    elif predicoes[i] == 1:
        print("Empate")
    elif predicoes[i] == 0:
        print("Vencedor da partida: " + backup_X2_teste.iloc[i,1])    
    print('Probabilidade de ' + backup_X2_teste.iloc[i,0] + ' vencer: ', '%.3f'%(lr.predict_proba(X2_teste_predicao)[i][2]))
    print('Probabilidade de empate', '%.3f'%(lr.predict_proba(X2_teste_predicao)[i][1]))
    print('Probabilidade de ' + backup_X2_teste.iloc[i,1] + ' vencer: ', '%.3f'%(lr.predict_proba(X2_teste_predicao)[i][0]))
    print("")

Grupo: Group A
Russia x Saudi Arabia
Vencedor da partida: Russia
Probabilidade de Russia vencer:  0.723
Probabilidade de empate 0.178
Probabilidade de Saudi Arabia vencer:  0.100

Grupo: Group A
Egypt x Uruguay
Vencedor da partida: Egypt
Probabilidade de Egypt vencer:  0.510
Probabilidade de empate 0.220
Probabilidade de Uruguay vencer:  0.270

Grupo: Group B
Morocco x Iran
Vencedor da partida: Morocco
Probabilidade de Morocco vencer:  0.439
Probabilidade de empate 0.310
Probabilidade de Iran vencer:  0.251

Grupo: Group B
Portugal x Spain
Vencedor da partida: Portugal
Probabilidade de Portugal vencer:  0.489
Probabilidade de empate 0.067
Probabilidade de Spain vencer:  0.444

Grupo: Group C
France x Australia
Vencedor da partida: France
Probabilidade de France vencer:  0.729
Probabilidade de empate 0.222
Probabilidade de Australia vencer:  0.050

Grupo: Group D
Argentina x Iceland
Vencedor da partida: Argentina
Probabilidade de Argentina vencer:  0.713
Probabilidade de empate 0.153
Pr

In [199]:
print(predicoes)

[2 2 2 2 2 2 2 0 2 2 2 2 2 1 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2
 2 0 0 2 0 2 2 1 0 2 2]


In [212]:
print (lr.predict_proba(X2_teste_predicao)[0][0])

0.09955976257470585
